In [1]:
from nba_api.stats.endpoints import leaguegamelog
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from termcolor import colored
import requests
import mysql.connector
import csv_getters.helper_functions as hf
from csv_getters.config import api_basketball_key, conn_host, conn_database, conn_user, conn_password
import datetime as dt
from IPython.display import clear_output
from dateutil import tz
import json
import time

In [2]:
season = '2022-2023'
now =  time.time()
date = dt.datetime.now().strftime('%Y-%m-%d')

min_games = 5
n_last_games = 15

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Sao_Paulo')

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

def execute_multiple_queries(queries):
    try:
        db = connect_to_db()
        mycursor = db.cursor()
        for query in queries:
            mycursor.execute(query)

        db.commit()
        db.close()
    except Exception as e:
        print(e)

In [4]:
def add_match_info_to_db(home_game, away_game, winner):
    query = (f"INSERT IGNORE INTO games (id, date, season, is_playoff, winner, " + 
                  "home_id, home_pts, home_fgm, home_fga, home_fg_pct, home_fg3m, home_fg3a, home_fg3_pct, home_ftm, home_fta, home_ft_pct, home_oreb, home_dreb, home_reb, home_ast, home_stl, home_blk, home_tov, home_pf," +
                  "away_id, away_pts, away_fgm, away_fga, away_fg_pct, away_fg3m, away_fg3a, away_fg3_pct, away_ftm, away_fta, away_ft_pct, away_oreb, away_dreb, away_reb, away_ast, away_stl, away_blk, away_tov, away_pf" +
                  f") VALUES ({home_game['GAME_ID']}, '{home_game['GAME_DATE']}', {get_season_year(home_game['SEASON_ID'])}, {home_game['IS_PLAYOFFS']}, '{winner}', " + 
                  f"{home_game['TEAM_ID']}, {home_game['PTS']}, {home_game['FGM']}, {home_game['FGA']}, {home_game['FG_PCT']}, {home_game['FG3M']}, {home_game['FG3A']}, {home_game['FG3_PCT']}, {home_game['FTM']}, {home_game['FTA']}, {home_game['FT_PCT']}, {home_game['OREB']}, {home_game['DREB']}, {home_game['REB']}, {home_game['AST']}, {home_game['STL']}, {home_game['BLK']}, {home_game['TOV']}, {home_game['PF']}, " +
                  f"{away_game['TEAM_ID']}, {away_game['PTS']}, {away_game['FGM']}, {away_game['FGA']}, {away_game['FG_PCT']}, {away_game['FG3M']}, {away_game['FG3A']}, {away_game['FG3_PCT']}, {away_game['FTM']}, {away_game['FTA']}, {away_game['FT_PCT']}, {away_game['OREB']}, {away_game['DREB']}, {away_game['REB']}, {away_game['AST']}, {away_game['STL']}, {away_game['BLK']}, {away_game['TOV']}, {away_game['PF']}" +
                  ")")
    return query

def add_player_to_db(player_id, name):
    query = (f"INSERT IGNORE INTO players (id, name) VALUES ({player_id}, '{name}')")
    return query
    
def add_player_game_to_db(game):
    query = (f"INSERT IGNORE INTO playergames (team_id, player_id, game_id, minutes, pts, fgm, fga, fg_pct, fg3m, fg3a, fg3_pct, ftm, fta, ft_pct, oreb, dreb, reb, ast, stl, blk, tov, pf, plus_minus) " + 
                  f"VALUES ({game['TEAM_ID']}, {game['PLAYER_ID']}, {game['GAME_ID']}, {game['MIN']}, {game['PTS']}, {game['FGM']}, {game['FGA']}, {game['FG_PCT']}, {game['FG3M']}, {game['FG3A']}, {game['FG3_PCT']}, {game['FTM']}, {game['FTA']}, {game['FT_PCT']}, {game['OREB']}, {game['DREB']}, {game['REB']}, {game['AST']}, {game['STL']}, {game['BLK']}, {game['TOV']}, {game['PF']}, {game['PLUS_MINUS']})")
    return query

def get_season(season, season_type='Regular Season'):
    season_games = leaguegamelog.LeagueGameLog(season = str(season), season_type_all_star=season_type).get_data_frames()[0]
    season_players = leaguegamelog.LeagueGameLog(season = str(season), player_or_team_abbreviation = 'P', season_type_all_star=season_type).get_data_frames()[0]
    season_games['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    season_players['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    
    season_games.dropna(subset=['FG_PCT','FT_PCT','FG3_PCT'], inplace=True)

    season_games['GAME_ID'] = pd.to_numeric(season_games['GAME_ID'])
    season_players['GAME_ID'] = pd.to_numeric(season_players['GAME_ID'])

    season_games['GAME_DATE'] = pd.to_datetime(season_games['GAME_DATE'])
    season_players['GAME_DATE'] = pd.to_datetime(season_players['GAME_DATE'])

    season_games = season_games.sort_values(['GAME_DATE', 'GAME_ID'], ascending=[True, True]).reset_index(drop=True)
    return season_games, season_players

def get_season_year(season_id):
    return int(str(season_id)[1:])

In [5]:
season_games, season_players = get_season(season.split('-')[0], season_type='Regular Season')

In [6]:
season_games.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,IS_PLAYOFFS
199,22022,1610612765,DET,Detroit Pistons,22200100,2022-10-31,DET @ MIL,L,240,41,...,43,19,9,3,12,24,108,-2,1,False
200,22022,1610612762,UTA,Utah Jazz,22200101,2022-10-31,UTA vs. MEM,W,240,42,...,48,26,10,6,16,30,121,16,1,False
201,22022,1610612763,MEM,Memphis Grizzlies,22200101,2022-10-31,MEM @ UTA,L,240,38,...,52,14,8,4,16,17,105,-16,1,False
202,22022,1610612746,LAC,LA Clippers,22200102,2022-10-31,LAC vs. HOU,W,240,37,...,46,23,7,10,14,18,95,2,1,False
203,22022,1610612745,HOU,Houston Rockets,22200102,2022-10-31,HOU @ LAC,L,240,34,...,44,17,9,8,15,15,93,-2,1,False


In [7]:
season_players.tail()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE,IS_PLAYOFFS
2260,22022,1627826,Ivica Zubac,1610612746,LAC,LA Clippers,22200102,2022-10-31,LAC vs. HOU,W,...,0,1,4,3,5,16,-2,42.4,1,False
2261,22022,1626181,Norman Powell,1610612746,LAC,LA Clippers,22200102,2022-10-31,LAC vs. HOU,W,...,0,0,1,0,1,9,6,13.2,1,False
2262,22022,202694,Marcus Morris Sr.,1610612746,LAC,LA Clippers,22200102,2022-10-31,LAC vs. HOU,W,...,3,0,1,1,0,11,7,21.1,1,False
2263,22022,202331,Paul George,1610612746,LAC,LA Clippers,22200102,2022-10-31,LAC vs. HOU,W,...,8,6,2,3,3,35,16,78.8,1,False
2264,22022,201569,Eric Gordon,1610612745,HOU,Houston Rockets,22200102,2022-10-31,HOU @ LAC,L,...,2,2,2,2,1,12,-10,27.4,1,False


In [8]:
games_to_insert_queries = []
players_to_insert_queries = []
player_games_to_insert_queries = []

season_id = ''
for i, g in season_games.groupby(season_games.index // 2):
    clear_output(wait=True)
    print("{}/{}".format(i, len(season_games.index) // 2))
    if g.iloc[[0],:].iloc[0]['WL'] == None:
        break
        
    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        away_game = g.iloc[0,:]
        home_game = g.iloc[1,:]
        winner = 'H' if g.iloc[1,:]['WL'] == 'W' else 'A'
    else:
        home_game = g.iloc[0,:]
        away_game = g.iloc[1,:]
        winner = 'H' if g.iloc[0,:]['WL'] == 'W' else 'A'
    
    game_players = season_players.loc[season_players['GAME_ID'] == home_game['GAME_ID']]
    game_players = game_players.replace({np.nan: 0})
    
    games_to_insert_queries.append(add_match_info_to_db(home_game, away_game, winner))
    
    for index, player in game_players.iterrows():
        players_to_insert_queries.append(add_player_to_db(player['PLAYER_ID'], player['PLAYER_NAME'].replace("'", "")))
        player_games_to_insert_queries.append(add_player_game_to_db(player))

101/102


In [9]:
execute_multiple_queries(games_to_insert_queries)
execute_multiple_queries(players_to_insert_queries)
execute_multiple_queries(player_games_to_insert_queries)

In [10]:
season_games = execute_query(f"SELECT g.id, g.date, g.season, g.is_playoff, g.winner, g.home_id, ht.name as home_name, g.home_pts, g.home_fgm, g.home_fga, g.home_fg_pct, g.home_fg3m, g.home_fg3a, g.home_fg3_pct, g.home_ftm, g.home_fta, g.home_ft_pct, g.home_oreb, g.home_dreb, g.home_reb, g.home_ast, g.home_stl, g.home_blk, g.home_tov, g.home_pf, g.away_id, at.name as away_name, g.away_pts, g.away_fgm, g.away_fga, g.away_fg_pct, g.away_fg3m, g.away_fg3a, g.away_fg3_pct, g.away_ftm, g.away_fta, g.away_ft_pct, g.away_oreb, g.away_dreb, g.away_reb, g.away_ast, g.away_stl, g.away_blk, g.away_tov, g.away_pf, g.home_odds, g.away_odds, g.over_under_line FROM games AS g LEFT JOIN teams as ht ON g.home_id = ht.id LEFT JOIN teams as at ON g.away_id = at.id WHERE g.season = {season.split('-')[0]} ORDER BY g.date ASC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
teams = execute_query("SELECT * FROM teams")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
teams.head()

,id,name,abbreviation
0,1610612737,Atlanta Hawks,ATL
1,1610612738,Boston Celtics,BOS
2,1610612739,Cleveland Cavaliers,CLE
3,1610612740,New Orleans Pelicans,NOP
4,1610612741,Chicago Bulls,CHI


In [13]:
def get_season_fixtures(season):
    headers = {
        'X-RapidAPI-Key': api_basketball_key,
        'X-RapidAPI-Host': 'api-basketball.p.rapidapi.com'
    }
    response = requests.get(f"https://api-basketball.p.rapidapi.com/games?league=12&season={season}&timezone=America/Sao_Paulo&date={date}", headers=headers)
    response_parsed = json.loads(response.text)
    return response_parsed['response']

In [14]:
fixtures = get_season_fixtures(season)
fixtures = [fixture for fixture in fixtures if fixture['timestamp'] >= now and fixture['timestamp'] <= (now + 24*60*60)]
fixtures = sorted(fixtures, key = lambda x: x['timestamp'])

In [15]:
def get_team_made_conceded_pct(games, line):
    made = np.array(games['team_pts'])
    conceded = np.array(games['opp_pts'])
    totals = made + conceded
    greater = (totals > line).sum()
    pct = greater*100/len(totals)
    return pct

def get_team_info(team_id, game_date, season, line, scenario):
    response = hf.get_team_previous_games(season_games, team_id, game_date, season)
    if not response: return None
    
    home_previous_games, away_previous_games, previous_games, previous_season_games = response
    
    if len(previous_season_games.index) < min_games:
        return None
    
    overall_pct = get_team_made_conceded_pct(previous_season_games.iloc[-n_last_games:,:], line)
    if scenario == 'H':
        ha_pct = get_team_made_conceded_pct(home_previous_games.iloc[-n_last_games:,:], line)
    else:
        ha_pct = get_team_made_conceded_pct(away_previous_games.iloc[-n_last_games:,:], line)
    return overall_pct, ha_pct

In [16]:
for fixture in fixtures:
    game_date = np.datetime64(dt.datetime.utcfromtimestamp(fixture['timestamp']).replace(tzinfo=from_zone).astimezone(to_zone).strftime('%Y-%m-%d %H:%M:%S'))
    home_name, away_name = fixture['teams']['home']['name'], fixture['teams']['away']['name']
    
    home_id = teams.loc[teams['name'] == home_name].iloc[0,0]
    away_id = teams.loc[teams['name'] == away_name].iloc[0,0]
    
    season_id = int(season.split('-')[0])
    
    print('\n')
    
    while True:
        line = input(f"Type the line that represents the totals line of the game {away_name} @ {home_name}: ").replace(',','.')
        try:
            line = float(line)
            break
        except ValueError:
            pass
        print("Invalid value.")
    
    response = get_team_info(home_id, game_date, season_id, line, 'H')
    if not response: continue

    home_overall_pct, home_ha_pct = response

    response = get_team_info(away_id, game_date, season_id, line, 'A')
    if not response: continue

    away_overall_pct, away_ha_pct = response
    
    print("\n{} had their totals above {} in {:.2f}% of the last {} games".format(home_name, line, home_overall_pct, n_last_games))
    print("At home, {} had their totals above {} in {:.2f}% of the last {} games".format(home_name, line, home_ha_pct, n_last_games))
    
    print("\n{} had their totals above {} in {:.2f}% of the last {} games".format(away_name, line, away_overall_pct, n_last_games))
    print("On the road, {} had their totals above {} in {:.2f}% of the last {} games".format(away_name, line, away_ha_pct, n_last_games))
    
    over = min(home_overall_pct, home_ha_pct, away_overall_pct, away_ha_pct) > 50
    under = max(home_overall_pct, home_ha_pct, away_overall_pct, away_ha_pct) < 50
    if over or under:
        print(colored(f"GOOD BET: {'OVER' if over else 'UNDER'}", 'green'))
    else:
        print(colored("BAD BET", 'red'))



Type the line that represents the totals line of the game Chicago Bulls @ Brooklyn Nets: 231,5

Brooklyn Nets had their totals above 231.5 in 57.14% of the last 15 games
At home, Brooklyn Nets had their totals above 231.5 in 60.00% of the last 15 games

Chicago Bulls had their totals above 231.5 in 28.57% of the last 15 games
On the road, Chicago Bulls had their totals above 231.5 in 33.33% of the last 15 games
BAD BET


Type the line that represents the totals line of the game Golden State Warriors @ Miami Heat: 227

Miami Heat had their totals above 227.0 in 28.57% of the last 15 games
At home, Miami Heat had their totals above 227.0 in 0.00% of the last 15 games

Golden State Warriors had their totals above 227.0 in 100.00% of the last 15 games
On the road, Golden State Warriors had their totals above 227.0 in 100.00% of the last 15 games
BAD BET


Type the line that represents the totals line of the game Orlando Magic @ Oklahoma City Thunder: 215

Oklahoma City Thunder had their 